In [20]:
import pandas as pd
import numpy as np
import csv
import difflib

### Step 1 : Read input Data

In [21]:
# PATH TO TARGET DATA
path_to_input_target = 'Data/GAP_Polypharmacy_Data.Students.csv'
target_drug_column = "drug_name"
target_patient_column = "Dummy id"
# PATH TO LIBRARY DATA
path_to_input_lib = "Data/medicare_medications_data_dictionary.csv"
lib_drug_column = "DRUG_NAME"
lib_drug_generic = "GENERIC_NAME"
lib_separator = "/"


In [3]:
"""
 read in and re-formatting target data

"""

dummy_id_dicts = csv.DictReader(open(path_to_input_target))
misspelled_names = []
patient_id =[]
trade_rec = []
for row in dummy_id_dicts:
    misspelled_names.append(row[target_drug_column])
    patient_id.append(row[target_patient_column])
    trade_rec.append((row[target_patient_column],row[target_drug_column]))

In [4]:
"""
 read in and re-formatting library data

"""

trade_vs_generic = csv.DictReader(open(path_to_input_lib))
trade_names = []
trade_vs_generic_dict={}
for row in trade_vs_generic:
    trade_names.append(row[lib_drug_column])
    generics = row[lib_drug_generic].split(lib_separator)
    trade_vs_generic_dict.update({row[lib_drug_column] : generics})
def dict_to_list(dict_name):
    return list(set([item for sublist in list(dict_name.values()) for item in sublist]))
generics_master_list=[]
generics_master_list.extend(dict_to_list(trade_vs_generic_dict))

### Step 2 : Clean by Generic Name Matching

In [6]:
from tqdm import tqdm
def check_generic_name(misspelled_names_list, name_dic,n,cutoff):
    name_list = []
    tmp_selection = []
    mark=np.array([])
    generic_list = []
    for i in tqdm(misspelled_names_list):
        if i in name_dic:
            name_list.append(i)
            generic_list.append([i])
            mark = np.append(mark,["GEN-Found"])
        else:
            tmp_selection = difflib.get_close_matches(i, name_dic,n,cutoff)
            if len(tmp_selection)>0:
                name_list.append(tmp_selection[0])
                generic_list.append([tmp_selection[0]])
                mark = np.append(mark,["GEN-Changed"])
            else:
                name_list.append(i)
                generic_list.append([])
                mark = np.append(mark,["Unknown"])
    return name_list, mark, generic_list

In [7]:
## INPUT FOR THRESHOLD
THRESHOLD = 0.7

GEN_checked_name_list,GEN_mark, generic_list= check_generic_name(misspelled_names,\
                                                                 generics_master_list,3,\
                                                                 THRESHOLD)

100%|██████████| 7667/7667 [00:37<00:00, 206.83it/s]


In [9]:
"""
 visualization and summary of result
 
"""

df_result = pd.DataFrame(list(zip(patient_id, misspelled_names,GEN_checked_name_list,GEN_mark)),columns =['patient_id', 'misspelled_names','GEN_checked_name_list','GEN_mark'])

df_changesReport = df_result.loc[df_result['GEN_mark'] == "GEN-Changed"]

report = pd.DataFrame({"Directly matching": df_result.loc[df_result['GEN_mark'] == "GEN-Found"].shape[0],
                       "Algorithm correction" : df_result.loc[df_result['GEN_mark'] == "GEN-Changed"].shape[0],
                       "Unknown value": df_result.loc[df_result['GEN_mark'] == "Unknown"].shape[0]},
                      index=[0])
report.head(2)


,Directly matching,Algorithm correction,Unknown value
0,2703,2541,2423


### Step 3 : Clean by Trade Name Matching

In [12]:
trade_names_master=trade_vs_generic_dict
def check_trade_name(misspelled_names_list, name_dic,n,cutoff,old_mark,old_generic_list):
    mark=np.array([])
    name_list = []
    generic_list = []
    for i in tqdm(range(len(old_mark))):
        if old_mark[i]=="Unknown":
            if misspelled_names_list[i] in list(name_dic.keys()):
                name_list.append(misspelled_names_list[i])
                generic_list.append(name_dic.get(misspelled_names_list[i]))
                mark = np.append(mark,["Found"])
            else:
                tmp_selection = difflib.get_close_matches(misspelled_names_list[i], list(name_dic.keys()),n,cutoff)
                if len(tmp_selection)>0:
                    name_list.append(tmp_selection[0])
                    generic_list.append(name_dic.get(tmp_selection[0]))
                    mark = np.append(mark,["Changed"])
                else:
                    name_list.append(misspelled_names_list[i])
                    generic_list.append([])
                    mark = np.append(mark,["Unknown"])
        else:
            name_list.append(misspelled_names_list[i])
            generic_list.append(old_generic_list[i])
            mark = np.append(mark,old_mark[i])
    return name_list, mark, generic_list

In [17]:
## INPUT FOR THRESHOLD
THRESHOLD = 0.7

checked_name_list,mark,new_generic_list = check_trade_name(GEN_checked_name_list,\
                                                           trade_names_master,\
                                                           3,THRESHOLD,\
                                                           GEN_mark,\
                                                           generic_list)

100%|██████████| 7667/7667 [00:16<00:00, 453.18it/s]


In [19]:
"""
    A display to show results.

    Variables:
      df_result: A dataframe of all patient ID vs corrected trade names.
      df_changesReport: A dataframe of corrected trade names at current cutoff rate.
      df_changesReportWarnings: A dataframe of possible mis-corrected trade names since having different first letter.
      report: A dataframe shows of all number of each variable contains above.
    Outputs:
      All dataframe file can be out put as csv file by using 
      DATAFRAME_NAME.to_csv('/PATH_TO_FILE/NAME.csv')
"""

df_result = pd.DataFrame(list(zip(patient_id, misspelled_names,checked_name_list,new_generic_list,mark)), 
               columns =['patient_id', 'misspelled_names','checked_name_list',"new_generic_list",'mark'])

df_totalCorrection = df_result.loc[df_result['mark'] != "Unknown"]
df_changesReport = df_result.loc[df_result['mark'] == "Changed"]
df_directedMatch = df_result.loc[df_result['mark'] == "Found"]
df_unknown = df_result.loc[df_result['mark'] == "Unknown"]

df_changesReportWarnings = df_changesReport.loc[df_changesReport['misspelled_names'].str[0] 
                                                != df_changesReport['checked_name_list'].str[0]] 

report = pd.DataFrame({"Total Correction Number": df_totalCorrection.shape[0],
                       "Directed matching": df_directedMatch.shape[0],
                       "Algorithm correction": df_changesReport.shape[0],
                       "Unknown value":df_unknown.shape[0],
                      "Warnings": df_changesReportWarnings.shape[0]},
                      index=["Times"])
report = report[['Total Correction Number', 'Directed matching', 'Algorithm correction','Unknown value','Warnings']]
report

,Total Correction Number,Directed matching,Algorithm correction,Unknown value,Warnings
Times,6709,998,467,958,114


In [ ]:
# PATH TO OUTPUT
path_to_output = ""
df_result.to_csv(path_to_output)